# Piper TTS Fine-tuning Plan

This notebook guides you through fine-tuning the `en_US-lessac-medium` model on your custom dataset using a T4 GPU.

## Prerequisites
1. **GPU Runtime**: Ensure you are connected to a T4 GPU (Runtime > Change runtime type > T4 GPU).
2. **Preprocessed Data**: You should have already run the preprocessing steps (generating `training_dir`).
3. **Piper Repository**: The `piper_repo` folder should be present and `monotonic_align` built (which you did in the previous notebook).

### 1. Check GPU Availability
First, verify that the T4 GPU is correctly assigned and visible.

In [1]:
!nvidia-smi

Wed Jan 14 08:24:43 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### 2. Setup Environment Variables
We need to point `PYTHONPATH` to the piper source code so Python can find the `piper_train` module.

In [5]:
import os

# Path to the local piper repository source
# Adjust this if your folder structure is different
piper_src_path = os.path.abspath("/Users/rutwik/piper-model-training/piper_repo/src/python")

print(f"Piper Source Path: {piper_src_path}")

# Verify that the path exists
if not os.path.exists(piper_src_path):
    print("WARNING: piper repository path not found! Check your directories.")

Piper Source Path: /Users/rutwik/piper-model-training/piper_repo/src/python


### 3. Download the Pre-trained Checkpoint
We will download the `en_US-lessac-medium` checkpoint to use as a starting point. This saves massive amounts of time compared to training from scratch.

In [ ]:
import urllib.request

# Create a folder for checkpoints
os.makedirs("checkpoints", exist_ok=True)

checkpoint_url = "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/en/en_US/lessac/medium/epoch%3D2164-step%3D1355540.ckpt"
checkpoint_path = "checkpoints/epoch=2164-step=1355540.ckpt"

print("Downloading checkpoint... this may take a minute.")
urllib.request.urlretrieve(checkpoint_url, checkpoint_path)
print(f"Checkpoint downloaded to: {checkpoint_path}")

### 4. Start Fine-tuning

**Key Parameters Explained:**
- `--dataset-dir`: Directory containing your preprocessed data (config.json, dataset.jsonl).
- `--accelerator 'gpu'`: Uses the T4 GPU.
- `--devices 1`: Uses 1 GPU.
- `--batch-size`: Set to `4` or `8` for T4 GPUs to avoid Out of Memory (OOM) errors.
- `--quality medium`: Must match the quality of the checkpoint we downloaded.
- `--resume_from_checkpoint`: Points to the file we just downloaded.
- `--checkpoint-epochs 1`: Saves a new checkpoint every epoch (good for monitoring).

**Note:** Ensure `training_dir` is the folder where your `config.json` and `dataset.jsonl` from the preprocessing step are located.

In [ ]:
!PYTHONPATH="{piper_src_path}" python3 -m piper_train \
  --dataset-dir training_dir \
  --accelerator 'gpu' \
  --devices 1 \
  --batch-size 4 \
  --validation-split 0.0 \
  --num-test-examples 0 \
  --max_epochs 2000 \
  --resume_from_checkpoint "{checkpoint_path}" \
  --checkpoint-epochs 1 \
  --quality medium \
  --precision 32